# 4houses using python-constraint

In [21]:
import constraint

problem = constraint.Problem()

problem.addVariable('A', range(1,5))
problem.addVariable('B', range(1,5))
problem.addVariable('C', range(1,5))
problem.addVariable('D', range(1,5))

problem.addConstraint(constraint.AllDifferentConstraint())

def c_housenumber_higher_then_d(c, d):
    if c>d:
        return True

def d_lives_next_to_a_with_lower_housenumber(a, d):
    if (d+1) == a:
        return True

def at_least_one_house_between_d_and_b(b, d):
    if abs(d-b) > 1:
        return True

def c_does_not_live_in_housenumber_3(c):
    if c != 3:
        return True
    
def b_does_not_live_in_housenumber_1(b):
    if b != 1:
        return True


problem.addConstraint(c_housenumber_higher_then_d, "CD")
problem.addConstraint(d_lives_next_to_a_with_lower_housenumber, "AD")
problem.addConstraint(at_least_one_house_between_d_and_b, "BD")
problem.addConstraint(c_does_not_live_in_housenumber_3, "C")
problem.addConstraint(b_does_not_live_in_housenumber_1, "B")

solution_found = {}
solutions = problem.getSolutionIter()


for s in solutions:
    print (f"Solution is: {s}")



Solution is: {'D': 1, 'A': 2, 'C': 4, 'B': 3}


# 4houses using backtracking


In [55]:
import constraint
from copy import deepcopy

def four_houses_problem():
    variables = ['A', 'B', 'C', 'D']
    domain = [1, 2, 3, 4]
    return (variables, domain)

def all_different_constraint(a,b,c,d):
    if (a!=b) and (b!=c) and (c!=d) and (a!=c) and (b!=d) and (a!=d):
        return True

def c_housenumber_higher_then_d(c, d):
    if c>d:
        return True

def d_lives_next_to_a_with_lower_housenumber(a, d):
    if (d+1) == a:
        return True

def at_least_one_house_between_d_and_b(b, d):
    if abs(d-b) > 1:
        return True

def c_does_not_live_in_housenumber_3(c):
    if c != 3:
        return True
    
def b_does_not_live_in_housenumber_1(b):
    if b != 1:
        return True
    
def check_completeness(solution, variables):
    return True if len(solution) == len(variables) else False

def static_variable_selection(solution, csp, debug=True):
    unassigned_variables = [x for x in csp[0] if x not in solution]
    variable = unassigned_variables[0]
    if debug: print (f"  Selected variable: {variable}")
    return variable

def static_domain_selection(variable, assignment, csp, debug=True):
    for domain_value in csp[1][variable]:
        if debug: print (f"  Selected value {domain_value} for {variable}")
        yield domain_value

def no_inference(csp, variable, value, assignment, debug=True):
    return True

def check_value_consistency(variable, value, assignment, debug=True):
    if assignment is None: return False
    
    #alldiff
    for a in assignment:
            if value is assignment[a]:
                if debug: print (f"    {variable} : {value} conflicts with {a} : {assignment[a]}")
                return False
            
    assignments = deepcopy(assignment)
    assignments[variable]=value
    
    if 'B' in assignments and not(b_does_not_live_in_housenumber_1(assignments['B'])):
        if debug: print ('b_does_not_live_in_housenumber_1')
        return False;
    if 'C' in assignments and not(c_does_not_live_in_housenumber_3(assignments['C'])): 
        if debug: print ('c_does_not_live_in_housenumber_3')
        return False;
    if 'C' in assignments and 'D' in assignments and not(c_housenumber_higher_then_d(assignments['C'], assignments['D'])): 
        if debug: print ('c_housenumber_higher_then_d')
        return False
    if 'D' in assignments and 'A' in assignments and \
    not(d_lives_next_to_a_with_lower_housenumber(assignments['A'], assignments['D'])): 
        if debug: print ('d_lives_next_to_a_with_lower_housenumber')
        return False
    if 'B' in assignments and 'D' in assignments and not(at_least_one_house_between_d_and_b(assignments['B'], assignments['D'])): 
        if debug: print ('at_least_one_house_between_d_and_b')
        return False
    
    return True


def backtracking_search(csp, params, debug=True):
    solution = dict()
    return backtrack(solution, csp, params, debug=debug)

def backtrack(assignment, csp, params, debug=True):
    select_unassigned_variable, order_domain_values, inference = params
    if debug: print (f"Solution: {assignment}")
    if check_completeness(assignment, csp[0]): return True, assignment
    variable = select_unassigned_variable(assignment, csp, debug=debug)
    for value in order_domain_values(variable, assignment, csp, debug):
        if check_value_consistency(variable, value, assignment, debug=debug):
            assignment[variable], m_csp = value, deepcopy(csp)
            if inference(csp, variable, value, assignment, debug=debug):
                result, assignment = backtrack(deepcopy(assignment), csp, params, debug=debug)
                if result: return True, assignment
            del assignment[variable]
            csp = m_csp
    return False, assignment

def solve(csp, params=(static_variable_selection, static_domain_selection, no_inference)):
    variables = csp[0]
    domains = dict([(x, csp[1]) for x in variables])
    result, solution = backtracking_search((variables, domains), params, debug=True)
    if not result: print ("No Solution")
    else: return solution    

In [56]:
solve(four_houses_problem())

Solution: {}
  Selected variable: A
  Selected value 1 for A
Solution: {'A': 1}
  Selected variable: B
  Selected value 1 for B
    B : 1 conflicts with A : 1
  Selected value 2 for B
Solution: {'A': 1, 'B': 2}
  Selected variable: C
  Selected value 1 for C
    C : 1 conflicts with A : 1
  Selected value 2 for C
    C : 2 conflicts with B : 2
  Selected value 3 for C
c_does_not_live_in_housenumber_3
  Selected value 4 for C
Solution: {'A': 1, 'B': 2, 'C': 4}
  Selected variable: D
  Selected value 1 for D
    D : 1 conflicts with A : 1
  Selected value 2 for D
    D : 2 conflicts with B : 2
  Selected value 3 for D
d_lives_next_to_a_with_lower_housenumber
  Selected value 4 for D
    D : 4 conflicts with C : 4
  Selected value 3 for B
Solution: {'A': 1, 'B': 3}
  Selected variable: C
  Selected value 1 for C
    C : 1 conflicts with A : 1
  Selected value 2 for C
Solution: {'A': 1, 'B': 3, 'C': 2}
  Selected variable: D
  Selected value 1 for D
    D : 1 conflicts with A : 1
  Selecte

{'A': 2, 'B': 3, 'C': 4, 'D': 1}